# Direct Preference Optimization (DPO) for YouTube Title Generation

## Environment Setup and Model Loading



### Package Installation and GPU Configuration



In this section, we will:

- Install **Unsloth** from GitHub:
  - `unsloth[colab-new]` from GitHub
- Install:
  - `xformers` (for Flash Attention / speedups)
  - `trl`, `peft`, `accelerate`, `bitsandbytes`, `triton`
- Install a few extra utilities:
  - `transformers`, `datasets`, `pandas`
- Verify that:
  - A **GPU** is available
  - Basic environment info (PyTorch, CUDA, GPU name) is correct


In [ ]:
# ================================================================
# Install Unsloth + Xformers + Core Libraries
#    - We install transformers, datasets, pandas explicitly.
# ================================================================

%%capture
# Install Unsloth from GitHub with the "colab-new" extra
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install xformers version compatible with the installed torch
from torch import __version__ as torch_version
from packaging.version import Version as V

xformers_version = "xformers==0.0.27" if V(torch_version) < V("2.4.0") else "xformers>=0.0.30"

# Install xformers + trl + peft + accelerate + bitsandbytes + triton
!pip install --no-deps {xformers_version} trl peft accelerate bitsandbytes triton

# Install commonly used extras
!pip install transformers datasets pandas


In [ ]:
# ================================================================
# Basic Environment and GPU Check
#    - Confirms PyTorch sees a GPU.
#    - Prints basic GPU and memory info.
# ================================================================

import torch

print("PyTorch version:", torch.__version__)
print("CUDA available :", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(device)
    total_mem_gb = torch.cuda.get_device_properties(device).total_memory / (1024**3)

    print(f"Using device   : {device}")
    print(f"GPU name       : {gpu_name}")
    print(f"Total VRAM     : {total_mem_gb:.2f} GB")

    free_mem, total_mem_bytes = torch.cuda.mem_get_info()
    free_mem_gb = free_mem / (1024**3)
    total_mem_gb2 = total_mem_bytes / (1024**3)
    print(f"Free VRAM      : {free_mem_gb:.2f} GB / {total_mem_gb2:.2f} GB")
else:
    print("⚠️ No GPU detected. Please enable a GPU runtime.")


PyTorch version: 2.9.0+cu126
CUDA available : True
Using device   : cuda
GPU name       : NVIDIA A100-SXM4-80GB
Total VRAM     : 79.32 GB
Free VRAM      : 78.90 GB / 79.32 GB


In [ ]:
# ================================================================
# Optional: Small Performance Tweaks
#    - Enable TF32 for matmul/cuDNN on Ampere GPUs (e.g., A100).
#    - Enable cuDNN benchmark.
# ================================================================

if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("Enabled TF32 for matmul and cuDNN (recommended on A100).")

torch.backends.cudnn.benchmark = True
print("cuDNN benchmark set to:", torch.backends.cudnn.benchmark)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Default DEVICE  :", DEVICE)


Enabled TF32 for matmul and cuDNN (recommended on A100).
cuDNN benchmark set to: True
Default DEVICE  : cuda


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


### Dataset Loading and Exploration

In this section, we will:

- Load the `"EliasHossain/youtube-titles-dpo"` dataset using Hugging Face `datasets`.
- Inspect:
  - Available **splits** (`train`, `valid`, …).
  - **Columns**: `prompt`, `chosen`, `rejected`.
- Look at a few examples to see:
  - How `prompt` is formatted (chat-style list with `role`/`content`).
  - What `chosen` vs `rejected` titles look like.
- Compute a few simple statistics on text lengths for context.


In [ ]:
# ================================================================
# Load the YouTube Titles DPO Dataset
# ================================================================

from datasets import load_dataset

dataset = load_dataset("EliasHossain/youtube-titles-dpo")
print(dataset)


README.md:   0%|          | 0.00/822 [00:00<?, ?B/s]

data/train-00000-of-00001-76f6a471166630(…):   0%|          | 0.00/39.2k [00:00<?, ?B/s]

data/valid-00000-of-00001-a3be3f52cf9748(…):   0%|          | 0.00/14.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1026 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1026
    })
    valid: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 114
    })
})


In [ ]:
# ================================================================
# Inspect Splits and Choose Main Split
# ================================================================

available_splits = list(dataset.keys())
print("Available splits:", available_splits)

if "train" in dataset:
    main_split_name = "train"
else:
    main_split_name = available_splits[0]

print("Using main split for exploration:", main_split_name)

main_split = dataset[main_split_name]
print(main_split)


Available splits: ['train', 'valid']
Using main split for exploration: train
Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1026
})


In [ ]:
# ================================================================
# Inspect Columns and One Example Row
#    - Note: prompt/chosen/rejected are *lists of messages*
#      in chat format: [{'role': 'user'/'assistant', 'content': '...'}]
# ================================================================

column_names = main_split.column_names
print("Column names:", column_names)

example_idx = 0
example = main_split[example_idx]

print(f"\nExample row at index {example_idx}")
for key, value in example.items():
    print(f"- {key}: {value}")


Column names: ['prompt', 'chosen', 'rejected']

Example row at index 0
- prompt: [{'content': 'Given the YouTube video idea write an engaging title.\n\n**Video Idea**: p-values. definition, examples, and misconceptions\n\n**Additional Guidance**:\n- Title should be between 30 and 75 characters long\n- Only return the title idea, nothing else!', 'role': 'user'}]
- chosen: [{'content': 'P-Values Decoded: Definitions, Examples, and Common Mistakes', 'role': 'assistant'}]
- rejected: [{'content': 'P-Values 101: Definitions, Examples, and Common Misunderstandings', 'role': 'assistant'}]


In [ ]:
# ================================================================
# Small Table of Examples (First 5 Rows)
#    - We show only the content fields for readability.
# ================================================================

import pandas as pd

num_rows_to_show = 5
subset = main_split.select(range(num_rows_to_show))

rows = []
for row in subset:
    def extract_first_content(msg_list):
        if isinstance(msg_list, list) and len(msg_list) > 0:
            return msg_list[0].get("content", "")
        return ""

    rows.append({
        "prompt_content":   extract_first_content(row["prompt"]),
        "chosen_content":   extract_first_content(row["chosen"]),
        "rejected_content": extract_first_content(row["rejected"]),
    })

df_preview = pd.DataFrame(rows)
df_preview


,prompt_content,chosen_content,rejected_content
0,Given the YouTube video idea write an engaging...,"P-Values Decoded: Definitions, Examples, and C...","P-Values 101: Definitions, Examples, and Commo..."
1,Given the YouTube video idea write an engaging...,How SHAP Values Can Improve Your ML Models,SHAP Values: The Missing Link in ML Interpreta...
2,Given the YouTube video idea write an engaging...,Unlocking Multimodal AI: A Beginner's Guide,How Multimodal AI Combines Text & Images
3,Given the YouTube video idea write an engaging...,Missing Data? 3 Easy Techniques You Need to Know,4 Steps to Fix Missing Data in Your Dataset
4,Given the YouTube video idea write an engaging...,How Transformers Revolutionize NLP in 5 Minutes,Transformers vs RNNs: What's the Difference?


In [ ]:
# ================================================================
# Basic Dataset Statistics (Properly Using `content`)
# ================================================================

from statistics import mean

num_examples = len(main_split)
print(f"Number of examples in '{main_split_name}' split:", num_examples)

def avg_char_length_msg_column(column_name, num_samples=1000):
    """Compute avg character length of the `.content` of the first message."""
    if column_name not in main_split.column_names:
        return None

    n = min(num_samples, len(main_split))
    subset = main_split.select(range(n))

    lengths = []
    for msg_list in subset[column_name]:
        if isinstance(msg_list, list) and len(msg_list) > 0:
            content = msg_list[0].get("content", "")
            lengths.append(len(content))

    if not lengths:
        return None
    return mean(lengths)

for col in ["prompt", "chosen", "rejected"]:
    avg_len = avg_char_length_msg_column(col)
    if avg_len is not None:
        print(f"Average content length of '{col}' (chars): {avg_len:.1f}")
    else:
        print(f"Column '{col}' not found or empty, skipping.")


Number of examples in 'train' split: 1026
Average content length of 'prompt' (chars): 222.2
Average content length of 'chosen' (chars): 46.4
Average content length of 'rejected' (chars): 47.3


#### Notes on the Preference Data Format

- Each of `prompt`, `chosen`, and `rejected` is stored as a **chat-style message list**:
  - Example: `[{ "role": "user", "content": "..." }]`
- For our purposes:
  - `prompt` is the **user instruction/context** for the video.
  - `chosen` is the **preferred** (better) title from the assistant.
  - `rejected` is the **less preferred** title from the assistant.
- Each row can be read as:
  > Given this `prompt`, `chosen` should be preferred over `rejected`.

This is exactly the structure needed for **Direct Preference Optimization (DPO)** in later sections.


### 1.4.1.3 Model and Tokenizer Setup (Qwen3-14B + Unsloth + 4-bit)

In this section, following the TA’s pattern, we will:

- Import `FastLanguageModel` from Unsloth.
- Set the **exact quantized model name** (to be filled in from the assignment).
- Load the model and tokenizer with:
  - `max_seq_length = 2048`
  - `load_in_4bit = True`
  - `load_in_8bit = False`
  - `full_finetuning = False` (we’ll use LoRA later instead)
- Ensure `pad_token` is set correctly (same as `eos_token`).
- Run a small **sanity generation** to confirm everything works.




In [ ]:
# ================================================================
# Import Unsloth and Set Model Name
# ================================================================

from unsloth import FastLanguageModel
import torch

# Using Qwen2.5-14B-Instruct model
model_name = "Qwen/Qwen2.5-14B-Instruct"

print("Using model_name:", model_name)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Using model_name: Qwen/Qwen2.5-14B-Instruct


In [ ]:
# ================================================================
# Load Model and Tokenizer with Unsloth (4-bit)
#    - Configuration:
#        - load_in_4bit = True
#        - load_in_8bit = False
#        - full_finetuning = False (we'll do LoRA later)
# ================================================================

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name       = model_name,
    max_seq_length   = 2048,     # can be increased later if needed
    load_in_4bit     = True,     # efficient memory usage
    load_in_8bit     = False,    # keep False if using 4-bit
    full_finetuning  = False,    # we'll use parameter-efficient fine-tuning
    # token          = "hf_...", # only needed for gated/private models
)

print("Model and tokenizer loaded successfully.")
print("Model type    :", type(model))
print("Tokenizer type:", type(tokenizer))


==((====))==  Unsloth 2025.11.4: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Model and tokenizer loaded successfully.
Model type    : <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'>
Tokenizer type: <class 'transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast'>


In [ ]:
# ================================================================
# Tokenizer Configuration
#    - Ensure pad_token is set properly.
#    - We set pad_token = eos_token if needed.
# ================================================================

# Some models already have eos_token set; we just make sure.
if tokenizer.eos_token is None:
    # If your particular model needs a specific EOS, adjust here.
    # Often, Unsloth/Qwen models already define eos_token.
    print("⚠️ tokenizer.eos_token is None; please set appropriately if needed.")
else:
    print("EOS token:", tokenizer.eos_token, "| id:", tokenizer.eos_token_id)

# Set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

print("PAD token set to:", tokenizer.pad_token, "| id:", tokenizer.pad_token_id)
print("Special tokens map:", tokenizer.special_tokens_map)


EOS token: <|im_end|> | id: 151645
PAD token set to: <|im_end|> | id: 151645
Special tokens map: {'eos_token': '<|im_end|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}


In [ ]:
# ================================================================
# Sanity Check: Simple Generation
#    - Verify that:
#        * Tokenization works
#        * Model can generate a short completion
# ================================================================

model.eval()

test_prompt = "Given the YouTube video idea about learning Python for beginners, write an engaging title."

inputs = tokenizer(
    test_prompt,
    return_tensors = "pt",
    padding = True,
    truncation = True,
).to(DEVICE)

print("Tokenized input keys:", inputs.keys())
print("Input shape:", inputs["input_ids"].shape)

with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens = 32,
        do_sample      = True,
        top_p          = 0.9,
        temperature    = 0.7,
    )

generated_text = tokenizer.decode(
    generated_ids[0],
    skip_special_tokens = True,
)

print("\n=== Sanity Check: Model Generation ===")
print("Prompt:")
print(test_prompt)
print("\nModel response:")
print(generated_text)


Tokenized input keys: KeysView({'input_ids': tensor([[22043,   279, 13370,  2766,  4522,   911,  6832, 13027,   369, 46850,
            11,  3270,   458, 22570,  2265,    13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')})
Input shape: torch.Size([1, 16])

=== Sanity Check: Model Generation ===
Prompt:
Given the YouTube video idea about learning Python for beginners, write an engaging title.

Model response:
Given the YouTube video idea about learning Python for beginners, write an engaging title. "Python for Beginners: Start Coding with Confidence in Just a Few Easy Steps!"


#### Summary of 1.4.1

We have now:

1. **Installed and configured the environment** following the TA’s style:
   - Unsloth from GitHub (`unsloth[colab-new]`), `xformers`, `trl`, `peft`, `accelerate`, `bitsandbytes`, `triton`.
2. **Loaded and explored** the `"EliasHossain/youtube-titles-dpo"` dataset:
   - Confirmed `prompt`, `chosen`, `rejected` as chat-style messages.
3. **Loaded the Qwen3-14B quantized model** with Unsloth:
   - 4-bit quantization (`load_in_4bit=True`)
   - `full_finetuning=False` (we’ll use LoRA later)
   - `pad_token` aligned with `eos_token`
   - A sanity generation confirms the model is working correctly.

Next up: **1.4.2 – LoRA Configuration and Base Model Evaluation**.


## LoRA Configuration and Base Model Testing

In this section, we will:

1. **Configure LoRA adapters** on top of the quantized Qwen2.5-14B-Instruct model using Unsloth:
   - Rank `r = 32`
   - Appropriate `lora_alpha`
   - Target attention + MLP modules:
     - `q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`
   - Verify how many parameters become trainable vs. full model size.

2. **Evaluate the base model (with freshly attached LoRA, before training)**:
   - Build a small helper to format prompts.
   - Generate YouTube titles for a few examples from the **validation** split.
   - Compare with `chosen` and `rejected` titles to understand baseline behavior.


In [ ]:
# ================================================================
# LoRA Adapter Configuration
#    - We wrap the existing model with LoRA using Unsloth.
#    - This lets us train a small number of additional parameters
#      instead of full fine-tuning the entire 14B model.
# ================================================================

from unsloth import FastLanguageModel

# LoRA hyperparameters
lora_r = 32            # Rank of the LoRA matrices (bottleneck size)
lora_alpha = 64        # Scaling factor for the LoRA updates
lora_dropout = 0.0     # 0 is typically best + optimized in Unsloth

# Target modules: attention projections and MLP projections
target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
]

print("Configuring LoRA with:")
print(f"- r            = {lora_r}")
print(f"- lora_alpha   = {lora_alpha}")
print(f"- lora_dropout = {lora_dropout}")
print(f"- target_modules = {target_modules}")

# Apply LoRA using Unsloth's helper
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_r,
    target_modules = target_modules,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,    # 0 is optimized
    bias = "none",                  # "none" is optimized
    use_gradient_checkpointing = "unsloth",  # Saves VRAM, good for long sequences
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("LoRA adapters added successfully.")


Configuring LoRA with:
- r            = 32
- lora_alpha   = 64
- lora_dropout = 0.0
- target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']


Unsloth 2025.11.4 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


LoRA adapters added successfully.


In [ ]:
# ================================================================
# Parameter Count and Savings
#    - We compute:
#       * Total number of parameters in the model.
#       * Number of trainable parameters (mostly LoRA).
#    - This shows how much we save vs. full fine-tuning.
# ================================================================

def count_parameters(model):
    total_params = 0
    trainable_params = 0
    for p in model.parameters():
        num = p.numel()
        total_params += num
        if p.requires_grad:
            trainable_params += num
    return total_params, trainable_params

total_params, trainable_params = count_parameters(model)

total_params_m = total_params / 1e6
trainable_params_m = trainable_params / 1e6
trainable_ratio = trainable_params / total_params * 100

print(f"Total parameters       : {total_params_m:.2f}M")
print(f"Trainable parameters   : {trainable_params_m:.2f}M")
print(f"Trainable ratio        : {trainable_ratio:.2f}%")

# Rough memory estimate (assuming 2 bytes per parameter for fp16-style)
# Note: This is just an *approximation* for the trainable weights.
approx_trainable_mem_mb = trainable_params * 2 / (1024**2)
print(f"Approx. memory for trainable (LoRA) params: {approx_trainable_mem_mb:.1f} MB")


Total parameters       : 8757.76M
Trainable parameters   : 137.63M
Trainable ratio        : 1.57%
Approx. memory for trainable (LoRA) params: 262.5 MB


#### LoRA Parameter & Memory Savings (Commentary)

- The **total parameters** reflect the full Qwen2.5-14B-Instruct model size.
- With LoRA, only about **`1.57%`** (from the printout above) of all parameters are **trainable**:
  - Trainable params ≈ **LoRA adapter weights only**
  - Base model weights stay **frozen** (and quantized in 4-bit).
- This gives two major benefits:
  1. **Memory savings**: We only store gradients/optimizer states for the small LoRA matrices.
  2. **Compute savings**: Backpropagation only flows through LoRA parameters, not the entire 14B model.

In practice, this means we can:
- Fine-tune a 14B model on a single A100 GPU.
- Use reasonable batch sizes and sequence lengths without running out of memory.


### Base Model Evaluation (Pre-DPO, Pre-LoRA-Training)

We will:

- Create a helper to extract & format prompts from the dataset.
- Use the **validation** split of `"EliasHossain/youtube-titles-dpo"`.
- Generate titles with the current model.
- Compare them to `chosen` and `rejected` titles for a small sample.


In [ ]:
# ================================================================
# Helper: Extract Prompt / Chosen / Rejected Text
#    - Remember: dataset stores them as lists of messages:
#        [{ "role": "...", "content": "..." }]
# ================================================================

valid_split = dataset["valid"]

def extract_first_content(msg_list):
    """Extract the `content` field from the first message in a list."""
    if isinstance(msg_list, list) and len(msg_list) > 0:
        return msg_list[0].get("content", "")
    return ""

# Quick sanity check on the validation set structure
example_valid = valid_split[0]
print("Validation example keys:", example_valid.keys())
print("Prompt content:\n", extract_first_content(example_valid["prompt"]))
print("\nChosen title:\n", extract_first_content(example_valid["chosen"]))
print("\nRejected title:\n", extract_first_content(example_valid["rejected"]))


Validation example keys: dict_keys(['prompt', 'chosen', 'rejected'])
Prompt content:
 Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!

Chosen title:
 Independent Component Analysis: What It Is and Why It Matters

Rejected title:
 Breakdown: Independent Component Analysis for Beginners


In [ ]:
# ================================================================
# Helper: Format Prompt for the Model (Chat-Aware if Possible)
#    - If the tokenizer has a chat template, use it.
#    - Otherwise, just pass the prompt text as-is.
# ================================================================

def format_prompt_for_model(prompt_text: str) -> str:
    """
    Given a raw user prompt text, format it appropriately for the model.
    If the tokenizer has `apply_chat_template`, we use it with a single
    user message. Otherwise, we fall back to plain text.
    """
    messages = [{"role": "user", "content": prompt_text}]

    # Many instruct/chat models (including Qwen variants) provide a chat template.
    if hasattr(tokenizer, "apply_chat_template") and tokenizer.chat_template is not None:
        return tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True,
        )
    else:
        # Fallback: return the user text directly
        return prompt_text


In [ ]:
# ================================================================
# Generate Sample Titles from Validation Prompts
# ================================================================

model.eval()

num_samples = 3  # You can change this to see more/less examples

for idx in range(num_samples):
    row = valid_split[idx]

    prompt_text    = extract_first_content(row["prompt"])
    chosen_title   = extract_first_content(row["chosen"])
    rejected_title = extract_first_content(row["rejected"])

    # Format prompt for the model (chat-style if supported)
    model_input_text = format_prompt_for_model(prompt_text)

    # Tokenize
    inputs = tokenizer(
        model_input_text,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = 2048,
    ).to(DEVICE)

    # Generate a short output
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens = 32,     # Titles are short
            do_sample      = True,
            top_p          = 0.9,
            temperature    = 0.7,
        )

    generated_text = tokenizer.decode(
        generated_ids[0],
        skip_special_tokens = True,
    )

    print("=" * 80)
    print(f"Example #{idx}")
    print("-" * 80)
    print("PROMPT:")
    print(prompt_text)
    print("\nCHOSEN (preferred) TITLE:")
    print(chosen_title)
    print("\nREJECTED TITLE:")
    print(rejected_title)
    print("\nBASE MODEL GENERATED TITLE:")
    print(generated_text)
    print()  # extra newline for spacing

print("=" * 80)


Example #0
--------------------------------------------------------------------------------
PROMPT:
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!

CHOSEN (preferred) TITLE:
Independent Component Analysis: What It Is and Why It Matters

REJECTED TITLE:
Breakdown: Independent Component Analysis for Beginners

BASE MODEL GENERATED TITLE:
system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!
assistant
Demystifying Independent Component Analysis: A Beginner's Guide

Example #1
--------------------------------------------------------------------------------
P

#### Baseline Observations (Before DPO Training)

After inspecting the `df_base_eval` table above (a few examples), we can qualitatively note:

- The **base Qwen2.5-14B-Instruct model**:
  - Usually produces **fluent and relevant** titles.
  - Sometimes **repeats part of the instructions** or includes extra explanatory text.
  - May not **strictly follow** the “only return the title” style (depending on the prompt and chat formatting).
- Compared to the **`chosen`** titles:
  - The base model’s titles can be **slightly more generic** or less optimized for click-worthiness.
  - DPO will help align the model more closely with human preferences reflected in `chosen` vs `rejected`.

These observations form our **baseline**.  
Later, after DPO training, we will:
- Re-run a similar evaluation.
- Compare how the titles change in terms of:
  - Adherence to instructions (only the title).
  - Engagement / specificity.
  - Similarity to human-preferred `chosen` titles.


## DPO Training Implementation

### DPO Training Configuration

In this section, we will:

- Configure a **`DPOConfig`** (from TRL) with:
  - Learning rate
  - Batch size and gradient accumulation
  - Number of epochs
  - Evaluation & saving strategy
  - Logging and monitoring settings
- Choose hyperparameters that make sense for:
  - A **14B** model with **LoRA + 4-bit**
  - A relatively small dataset (~1k training examples)
  - An A100 GPU (good memory, good speed)

We will **not** start training yet — just set up the configuration object.


In [ ]:
# ================================================================
# DPO Training Configuration
#    - We use TRL's DPOConfig (a TrainingArguments subclass).
#    - This controls how long we train, batch sizes, logging, etc.
# ================================================================

from trl import DPOConfig
import torch

# ------------------------------
# Device / precision decision
# ------------------------------
# On an A100, bfloat16 (bf16) is usually well-supported and stable.
# We enable it if available. Otherwise, you could fall back to fp16.
sm_major, sm_minor = torch.cuda.get_device_capability() if torch.cuda.is_available() else (0, 0)
use_bf16 = torch.cuda.is_available() and (sm_major >= 8)  # Ampere+ (A100 etc.)

print(f"GPU compute capability: {sm_major}.{sm_minor}")
print("Using bf16 for training:", use_bf16)

# ------------------------------
# High-level training hyperparameters
# ------------------------------
# Dataset:
# - ~1026 train examples
# Strategy:
# - Small per-device batch size + gradient accumulation
#   so we get a reasonable *effective* batch without OOM.
per_device_train_batch_size = 4
gradient_accumulation_steps = 4   # Effective batch size ~ 4 * 4 = 16
num_train_epochs = 3              # You can justify 2–3 in the report

learning_rate = 5e-5              # Typical for LoRA on big models
warmup_ratio = 0.1                # 10% warmup to stabilize early training

# Logging and saving:
logging_steps = 10
eval_strategy = "epoch"           # Evaluate once per epoch
save_strategy = "epoch"           # Save at the end of each epoch

output_dir = "./qwen2_5_14b_youtube_dpo"  # Where checkpoints/logs will go

# ------------------------------
# DPO-specific hyperparameters
# ------------------------------
# beta controls how "sharp" the preference is.
# Common values in the wild: 0.1, 0.2, 0.5.
dpo_beta = 0.1

print("\nDPO beta parameter:", dpo_beta)

# ------------------------------
# Create the DPOConfig
# ------------------------------
dpo_config = DPOConfig(
    output_dir = output_dir,
    per_device_train_batch_size = per_device_train_batch_size,
    per_device_eval_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    learning_rate = learning_rate,
    num_train_epochs = num_train_epochs,
    lr_scheduler_type = "cosine",   # Smooth schedule
    warmup_ratio = warmup_ratio,

    logging_steps = logging_steps,
    eval_strategy = eval_strategy,
    save_strategy = save_strategy,
    save_total_limit = 2,           # Keep only last few checkpoints

    bf16 = use_bf16,
    fp16 = False,                   # We'll prefer bf16 on A100
    optim = "paged_adamw_8bit",     # bitsandbytes optimizer (memory efficient)
    max_grad_norm = 1.0,

    # TRL/Trainer-specific niceties
    report_to = "none",             # Set to "tensorboard" or "wandb" if you want
    remove_unused_columns = False,  # Important for TRL-style trainers

    # DPO-specific
    beta = dpo_beta,
)

print("\nDPOConfig created.")

GPU compute capability: 8.0
Using bf16 for training: True

DPO beta parameter: 0.1

DPOConfig created.


#### Notes on DPO Configuration Choices

- **Batching & epochs**
  - `per_device_train_batch_size = 4`, `gradient_accumulation_steps = 4`  
    → Effective batch size ≈ **16**, which is reasonable for a 14B model with LoRA on an A100.
  - `num_train_epochs = 3` over ~1026 examples gives enough passes to learn the preference signal without extreme overfitting.

- **Learning rate & schedule**
  - `learning_rate = 5e-5` is a common choice for **LoRA fine-tuning** on large models.
  - `lr_scheduler_type = "cosine"` + `warmup_ratio = 0.1`:
    - Gradually increases LR during the first 10% of steps.
    - Then decays smoothly, which tends to be stable.

- **Precision & optimizer**
  - `bf16 = True` (on A100) uses **bfloat16**, which is:
    - More numerically stable than fp16 in many cases.
    - Efficient on Ampere GPUs.
  - `optim = "paged_adamw_8bit"`:
    - Uses a memory-efficient 8-bit AdamW from bitsandbytes.
    - Good fit with 4-bit model loading + LoRA.

- **Evaluation & saving**
  - `evaluation_strategy = "epoch"` and `save_strategy = "epoch"`:
    - Evaluate and checkpoint once per epoch.
  - `save_total_limit = 2`:
    - Avoids disk clutter by keeping only the last few checkpoints.

- **DPO-specific**
  - `beta = 0.1` controls how strongly the model differentiates between
    **chosen** and **rejected** responses.  
    Lower values → softer preferences; higher values → sharper separation.

This configuration is now ready to be passed into `DPOTrainer` in the next step.


### DPO Trainer Setup and Execution

In this section, we will:

- Prepare the `"train"` and `"valid"` splits for DPO:
  - Convert `prompt`, `chosen`, and `rejected` from chat-style message lists → plain strings.
- Patch TRL’s `DPOTrainer` with Unsloth’s optimized implementation.
- Initialize `DPOTrainer` with:
  - LoRA-wrapped model
  - `dpo_config` (training hyperparameters)
  - Tokenizer
  - Train and eval datasets
- Run `.train()` with basic error handling and inspect training metrics.
- Save the trained model checkpoint for later evaluation.


In [ ]:
# ================================================================
# Prepare Train / Valid Datasets for DPO
#    - Convert from chat-style messages to plain text fields:
#        prompt:   "Given the YouTube video idea..."
#        chosen:   "Preferred title..."
#        rejected: "Less preferred title..."
#    - Keep column names as "prompt", "chosen", "rejected"
#      so DPOTrainer can use them directly.
# ================================================================

def to_dpo_format(batch):
    """Map chat-style message lists to plain text strings for DPO."""
    prompts = []
    chosens = []
    rejecteds = []

    for p_msgs, c_msgs, r_msgs in zip(batch["prompt"], batch["chosen"], batch["rejected"]):
        prompts.append(extract_first_content(p_msgs))
        chosens.append(extract_first_content(c_msgs))
        rejecteds.append(extract_first_content(r_msgs))

    return {
        "prompt": prompts,
        "chosen": chosens,
        "rejected": rejecteds,
    }

# Original splits: chat-style structure
train_raw = dataset["train"]
valid_raw = dataset["valid"]

# Convert to DPO-style string columns
train_dpo = train_raw.map(
    to_dpo_format,
    batched = True,
    remove_columns = train_raw.column_names,  # keep only new ones
)

valid_dpo = valid_raw.map(
    to_dpo_format,
    batched = True,
    remove_columns = valid_raw.column_names,
)

print("Train DPO dataset example:")
print(train_dpo[0])
print("\nValid DPO dataset example:")
print(valid_dpo[0])


Map:   0%|          | 0/1026 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Train DPO dataset example:
{'prompt': 'Given the YouTube video idea write an engaging title.\n\n**Video Idea**: p-values. definition, examples, and misconceptions\n\n**Additional Guidance**:\n- Title should be between 30 and 75 characters long\n- Only return the title idea, nothing else!', 'chosen': 'P-Values Decoded: Definitions, Examples, and Common Mistakes', 'rejected': 'P-Values 101: Definitions, Examples, and Common Misunderstandings'}

Valid DPO dataset example:
{'prompt': 'Given the YouTube video idea write an engaging title.\n\n**Video Idea**: intro independent component analysis\n\n**Additional Guidance**:\n- Title should be between 30 and 75 characters long\n- Only return the title idea, nothing else!', 'chosen': 'Independent Component Analysis: What It Is and Why It Matters', 'rejected': 'Breakdown: Independent Component Analysis for Beginners'}


In [ ]:
# ================================================================
# Patch TRL's DPOTrainer with Unsloth's optimized version
# ================================================================

from unsloth import PatchDPOTrainer
PatchDPOTrainer()  # This monkey-patches TRL's DPOTrainer under the hood

from trl import DPOTrainer


In [ ]:
# ================================================================
# Initialize DPOTrainer
#    - We pass:
#        * model              : LoRA-wrapped Qwen2.5-14B
#        * args               : dpo_config (DPOConfig)
#        * train_dataset      : train_dpo
#        * eval_dataset       : valid_dpo
#        * processing_class   : tokenizer (for tokenization)
#        * beta               : dpo_config.beta (DPO loss parameter)
# ================================================================

dpo_trainer = DPOTrainer(
    model = model,
    args = dpo_config,
    beta = dpo_config.beta,
    train_dataset = train_dpo,
    eval_dataset = valid_dpo,
    processing_class = tokenizer,  # Unsloth/TRL uses "processing_class" for tokenizer/preprocessor
)

print("DPOTrainer initialized.")


Extracting prompt in train dataset (num_proc=16):   0%|          | 0/1026 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=16):   0%|          | 0/1026 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=16):   0%|          | 0/1026 [00:00<?, ? examples/s]

Extracting prompt in eval dataset (num_proc=16):   0%|          | 0/114 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=16):   0%|          | 0/114 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=16):   0%|          | 0/114 [00:00<?, ? examples/s]

DPOTrainer initialized.


In [ ]:
# ================================================================
# Run DPO Training
#    - We wrap .train() in try/except just in case.
#    - Training logs (loss, rewards, etc.) will appear in the notebook.
# ================================================================

train_result = None

try:
    print("Starting DPO training...")
    train_result = dpo_trainer.train()
    print("DPO training finished.")
except Exception as e:
    print("⚠️ Error during DPO training:")
    print(e)


The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting DPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,026 | Num Epochs = 3 | Total steps = 195
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 137,625,600 of 14,907,659,264 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.567300,0.533397,-0.526763,-1.262154,0.741379,0.735391,-54.571930,-65.612740,-1.172432,-1.167270,0,0,0
2,0.388000,0.504152,-1.685016,-3.154165,0.784483,1.469149,-66.154465,-84.532852,-1.572392,-1.568697,No Log,No Log,No Log
3,0.278600,0.559167,-2.041302,-3.888911,0.741379,1.847608,-69.717339,-91.880310,-1.784077,-1.781428,No Log,No Log,No Log


DPO training finished.


In [ ]:
# ================================================================
# Inspect Training Metrics
#    - HF/TRL returns a TrainOutput object with .metrics.
#    - We also run a final evaluation pass.
# ================================================================

if train_result is not None:
    print("\n=== Training Metrics ===")
    if hasattr(train_result, "metrics") and train_result.metrics is not None:
        for k, v in train_result.metrics.items():
            print(f"{k}: {v}")
    else:
        print("No metrics found in train_result.")

    # Run evaluation on the validation split
    print("\nRunning final evaluation on validation split...")
    eval_metrics = dpo_trainer.evaluate()

    print("\n=== Evaluation Metrics ===")
    for k, v in eval_metrics.items():
        print(f"{k}: {v}")
else:
    print("No train_result available; training may have failed earlier.")



=== Training Metrics ===
train_runtime: 833.9012
train_samples_per_second: 3.691
train_steps_per_second: 0.234
total_flos: 0.0
train_loss: 0.41513219246497524
epoch: 3.0

Running final evaluation on validation split...



=== Evaluation Metrics ===
eval_loss: 0.5591673851013184
eval_runtime: 13.1785
eval_samples_per_second: 8.65
eval_steps_per_second: 2.201
eval_rewards/chosen: -2.041302442550659
eval_rewards/rejected: -3.8889107704162598
eval_rewards/accuracies: 0.7413793206214905
eval_rewards/margins: 1.8476083278656006
eval_logps/chosen: -69.71733856201172
eval_logps/rejected: -91.88031005859375
eval_logits/chosen: -1.7840766906738281
eval_logits/rejected: -1.7814279794692993
epoch: 3.0


In [ ]:
# ================================================================
# Save Trained Model + Tokenizer
#    - We save to the same output_dir used in DPOConfig.
#    - This will store the LoRA adapters and tokenizer config.
# ================================================================

output_dir = dpo_config.output_dir

print(f"Saving model and tokenizer to: {output_dir}")
dpo_trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model and tokenizer saved.")


Saving model and tokenizer to: ./qwen2_5_14b_youtube_dpo
Model and tokenizer saved.


#### Observations from DPO Training

- **Training completed successfully**  
  - Trained for **3 epochs** with a final `train_loss ≈ 0.42`.  
  - Evaluation loss is slightly higher (`eval_loss ≈ 0.56`), which is normal but suggests some mild overfitting / distribution shift between train and valid.

- **Model clearly prefers chosen over rejected titles**  
  - `eval_rewards/chosen ≈ -2.04` vs `eval_rewards/rejected ≈ -3.89`  
    → Higher (less negative) rewards for **chosen** responses.  
  - `eval_rewards/margins ≈ 1.85`  
    → On average, chosen titles are rewarded significantly more than rejected ones.

- **Good preference accuracy**  
  - `eval_rewards/accuracies ≈ 0.74`  
    → In ~**74%** of validation pairs, the model assigns a higher reward (preference) to the **chosen** title than to the **rejected** one.  
    This indicates that the DPO training has successfully aligned the model to human preferences in the dataset.

- **Log-probabilities match the preference direction**  
  - `eval_logps/chosen ≈ -69.7` vs `eval_logps/rejected ≈ -91.9`  
    → Chosen titles are assigned **higher probability** (less negative log-prob) than rejected titles on average.  
  - This is exactly the behavior DPO is designed to enforce.

- **Runtime looks reasonable for a 14B model with LoRA**  
  - `train_runtime ≈ 834s` (~14 minutes) for 3 epochs over ~1K examples.  
  - This is consistent with a **LoRA + 4-bit** setup on an A100 and indicates the training loop is efficient/stable.


## Model Evaluation and Comparison

In this section, we will:

1. Compare the **base model** (before DPO) and the **DPO fine-tuned model** on the same validation prompts.
2. Qualitatively analyze the generated YouTube titles:
   - Relevance to the video idea
   - Engagement / click-worthiness
   - Adherence to the “only return the title” style
3. Connect these observations to the **DPO training process** and the **metrics** observed earlier.


In [ ]:
# ================================================================
# Load a Fresh Base Model (No LoRA, No DPO)
#    - Our current `model` is the DPO fine-tuned LoRA model.
#    - For a fair "before vs after" comparison, we load a separate
#      base model from the original checkpoint.
#    - We reuse the *same* tokenizer.
# ================================================================

from unsloth import FastLanguageModel

# Alias the fine-tuned model clearly
dpo_model = model          # DPO fine-tuned LoRA model
dpo_model.eval()

# Load a fresh base model in 4-bit (same model_name as before)
# NOTE: This will use extra VRAM. On an A100 with 4-bit + LoRA,
# it should still be okay. If you hit OOM, you can:
#   - Restart, run just this block and the comparison.
#   - Or delete dpo_model first and reload it later from checkpoint.
base_model, _ = FastLanguageModel.from_pretrained(
    model_name      = model_name,
    max_seq_length  = 2048,
    load_in_4bit    = True,
    load_in_8bit    = False,
    full_finetuning = False,
)

base_model.eval()

print("Base model and DPO model both loaded and ready for comparison.")


==((====))==  Unsloth 2025.11.4: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Base model and DPO model both loaded and ready for comparison.


In [ ]:
# ================================================================
# Helper: Generate a Title from a Prompt with a Given Model
#    - Only decode the *new* tokens after the prompt,
#      so we don't print the system/user part of the chat template.
# ================================================================

import torch

def generate_title_from_prompt(model, prompt_text: str, max_new_tokens: int = 32) -> str:
    """
    Format a prompt, tokenize it, generate a short title, and
    return ONLY the newly generated assistant text (no prompt/system).
    """
    # Reuse the same formatter used earlier (chat template if available)
    model_input_text = format_prompt_for_model(prompt_text)

    # Tokenize the *full* chat-formatted input
    inputs = tokenizer(
        model_input_text,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = 2048,
    ).to(DEVICE)

    input_ids = inputs["input_ids"]
    input_len = input_ids.shape[1]

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens = max_new_tokens,
            do_sample      = True,
            top_p          = 0.9,
            temperature    = 0.7,
        )

    # Slice off the prompt part: keep only tokens after the original input
    new_tokens = generated_ids[0, input_len:]

    # Decode only the new tokens
    generated_text = tokenizer.decode(
        new_tokens,
        skip_special_tokens = True,
    )

    # Clean up whitespace
    return generated_text.strip()


In [ ]:
# ================================================================
# Qualitative Comparison: Base vs DPO on Validation Set
#    - For a few validation examples:
#        * Print the original prompt
#        * Show chosen + rejected titles (from dataset)
#        * Show BASE model's generated title
#        * Show DPO model's generated title
# ================================================================

num_compare_samples = 5  # You can change this to see more examples

for idx in range(num_compare_samples):
    row = valid_split[idx]

    prompt_text    = extract_first_content(row["prompt"])
    chosen_title   = extract_first_content(row["chosen"])
    rejected_title = extract_first_content(row["rejected"])

    base_title = generate_title_from_prompt(base_model, prompt_text)
    dpo_title  = generate_title_from_prompt(dpo_model, prompt_text)

    print("=" * 100)
    print(f"Example #{idx}")
    print("-" * 100)
    print("PROMPT:")
    print(prompt_text)
    print("\nCHOSEN (preferred) TITLE:")
    print(chosen_title)
    print("\nREJECTED TITLE:")
    print(rejected_title)
    print("\nBASE MODEL GENERATED TITLE:")
    print(base_title)
    print("\nDPO MODEL GENERATED TITLE:")
    print(dpo_title)
    print()  # extra newline


Example #0
----------------------------------------------------------------------------------------------------
PROMPT:
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!

CHOSEN (preferred) TITLE:
Independent Component Analysis: What It Is and Why It Matters

REJECTED TITLE:
Breakdown: Independent Component Analysis for Beginners

BASE MODEL GENERATED TITLE:
Demystifying Independent Component Analysis: A Beginner's Guide

DPO MODEL GENERATED TITLE:
Introduction to Independent Component Analysis (ICA)

Example #1
----------------------------------------------------------------------------------------------------
PROMPT:
Given the YouTube video idea write an engaging title.

**Video Idea**: llm fine-tuning faq

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title 

### Analysis of Generated Titles: Engagement and Quality

Looking at the generated titles purely from a **YouTube/engagement** perspective:

- **Relevance & clarity**
  - Both models produce titles that clearly reflect the underlying idea and would make sense to a viewer browsing YouTube.
  - Titles usually highlight the main topic (“Independent Component Analysis”, “LLM Fine-tuning”, “Synthetic Data with LLMs”, etc.), which is good for clarity and searchability.

- **Hooks and framing**
  - The **base model** often uses generic but safe hooks like “A Beginner’s Guide”, “Your Ultimate FAQ Guide”, “What’s the Real Difference?”.
  - The **DPO model** tends to:
    - Lean into **FAQ / explanation** framing (“Frequently Asked Questions Answered”).
    - Emphasize **comparisons** or structured explanations (“Roles & Responsibilities Compared”, “Understanding Statistical Differences …”).
  - These choices are in line with the style of many `chosen` titles, which often highlight value (“Explained”, “Why It Matters”, “Like a Pro”).

- **Instruction-following**
  - After fixing decoding to return only the completion, both models:
    - Output **only a single title**, without extra paragraphs or meta-text.
    - Respect the approximate character range implied by the instructions.
  - This means the fine-tuning is mostly influencing **which kind of good title** the model prefers, not whether it follows the “title only” instruction.

In short, the DPO model roughly matches the base model’s quality but often picks a slightly more targeted, explanatory, or comparison-style framing that aligns well with engagement-oriented YouTube titling.


### Discussion of the DPO Training Process and Observed Improvements

The DPO training process for this task can be summarized as:

- I started from a **pretrained Qwen2.5-14B-Instruct** model loaded in 4-bit and wrapped it with **LoRA adapters** (about 1.57% of parameters trainable).
- Using the `"EliasHossain/youtube-titles-dpo"` dataset, each training example consisted of:
  - A `prompt` (YouTube video idea + instructions),
  - A `chosen` title (preferred),
  - A `rejected` title (less preferred).
- Instead of training a separate reward model and running PPO, DPO directly adjusts the model so that:
  - It assigns **higher probability** to `chosen` titles than to `rejected` titles, given the same prompt.
- Training was done for **3 epochs** with:
  - A modest effective batch size (via gradient accumulation),
  - An 8-bit optimizer (`paged_adamw_8bit`) and bfloat16 for efficiency on A100.

Observed improvements:

- Quantitatively, the model achieves a **preference accuracy** of around **74%** on the validation set (the chosen title gets higher reward/probability than the rejected one in most cases).
- Qualitatively, the DPO-tuned model:
  - Keeps the base model’s fluency and correctness.
  - Shows a consistent shift towards styles that reflect the preference data (FAQ framing, clearer comparisons, explanatory wording), without drastically changing the underlying meaning of titles.

So, DPO works here as a **lightweight alignment step**: it doesn’t reinvent the model, but it nudges it to more often prefer the kind of titles humans labeled as better.


### Documentation and Interpretation of Training Metrics

Key metrics from DPO training and evaluation:

- **Losses**
  - Final `train_loss ≈ 0.42`
  - Final `eval_loss  ≈ 0.56`
  - The slightly higher evaluation loss suggests the model has learned the preference structure while not heavily overfitting the training set (reasonable gap given ~1k train examples).

- **Preference rewards**
  - `eval_rewards/chosen ≈ -2.04`
  - `eval_rewards/rejected ≈ -3.89`
  - `eval_rewards/margins ≈ 1.85`
  - Because higher reward is “better” here, the chosen titles consistently receive higher rewards than rejected ones, with a meaningful average margin between them.

- **Preference accuracy**
  - `eval_rewards/accuracies ≈ 0.74`
  - This means that in about **74%** of validation pairs, the model assigns a higher reward (and effectively higher preference) to the `chosen` title than the `rejected` one.
  - It aligns with the qualitative observation that the DPO model is more often—but not always—stylistically closer to the chosen titles.

- **Log-probabilities**
  - `eval_logps/chosen ≈ -69.7` vs `eval_logps/rejected ≈ -91.9`
  - Chosen titles are assigned **higher probability** (less negative log-prob) than rejected titles on average, which is exactly what DPO is designed to enforce.

Overall, these metrics show that the DPO procedure successfully **reshaped the model’s preferences** in a statistically meaningful way (higher reward, higher log-prob, ~74% accuracy on chosen vs rejected), and the qualitative examples confirm that this change manifests as subtle but consistent stylistic shifts in the generated titles.
